In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/dataset/data/food_brand_only.csv')  # Ganti 'dataset.csv' dengan nama file dataset Anda
# Preprocessing data
# data['calories'] = data['calories'].apply(lambda x: x.replace(' kcal', '')).astype(float)
# data['fat'] = data['fat'].apply(lambda x: x.replace('g', '')).astype(float)
# data['sugar'] = data['sugar'].apply(lambda x: x.replace('g', '')).astype(float)

# Calculate calories per unit
data['calories_per_unit'] = data['calories'] / data['metric_serving_amount']

from sklearn.preprocessing import MinMaxScaler

# Normalize "metric_serving_amount" with consideration to calories per unit
scaler = MinMaxScaler()
data['metric_serving_amount_normalized'] = scaler.fit_transform(data[['metric_serving_amount']])

# Adjust calories based on the normalized serving amount
data['calorie_normalized'] = data['metric_serving_amount_normalized'] * data['calories_per_unit']

def determine_meal_type(row):
    if row['calories'] <= 300 :
        return 'Dinner'
    elif row['calories'] <= 500 :
        return 'Breakfast'
    else:
        return 'Lunch'

# Create 'meal_type' column
data['meal_type'] = data.apply(determine_meal_type, axis=1)

# Save updated dataset
data.to_csv('dataset_with_meal_type.csv', index=False)

# Split data into features and labels
X = data[['calorie_normalized']]
y = data[['meal_type', 'food_name','food_id','serving_id']]  # Menambahkan kolom 'food_name' sebagai label

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
X_train_norm = (X_train - X_train.mean()) / X_train.std()
X_test_norm = (X_test - X_train.mean()) / X_train.std()

# Convert labels to one-hot encoding
y_train_encoded = pd.get_dummies(y_train['meal_type'])
y_test_encoded = pd.get_dummies(y_test['meal_type'])

# Define model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_norm, y_train_encoded, epochs=50, batch_size=32, verbose=1)

# Evaluate model
loss, accuracy = model.evaluate(X_test_norm, y_test_encoded, verbose=0)
print(f'Accuracy: {accuracy}')



Epoch 1/50
3952/3952 [==============================] - 16s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 2/50
3952/3952 [==============================] - 11s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 3/50
3952/3952 [==============================] - 11s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 4/50
3952/3952 [==============================] - 11s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 5/50
3952/3952 [==============================] - 11s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 6/50
3952/3952 [==============================] - 10s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 7/50
3952/3952 [==============================] - 11s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 8/50
3952/3952 [==============================] - 11s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 9/50
3952/3952 [==============================] - 11s 3ms/step - loss: nan - accuracy: 0.1110
Epoch 10/50
3952/3952 [==============================] - 11s 3ms/step - loss: nan - accuracy: 0.1110

In [ ]:
# User input for maximum nutrient values per day
max_calorie = float(input("Enter maximum calorie per day: "))
# max_fat = float(input("Enter maximum fat per day: "))
# max_sugar = float(input("Enter maximum sugar per day: "))

# Normalize user input
user_input = pd.DataFrame([[max_calorie]], columns=['calorie_normalized'])
user_input_norm = (user_input - X_train.mean()) / X_train.std()

# Get meal recommendations
recommendations = model.predict(user_input_norm)
meal_types = y_train_encoded.columns.values

for i, recommendation in enumerate(recommendations[0]):
    print(f'Recommended {meal_types[i]}:')
    recommended_foods = y_train[(y_train['meal_type'] == meal_types[i])][['food_id', 'serving_id', 'food_name']]
    recommended_food_samples = recommended_foods.sample(n=5)
    for index, row in recommended_food_samples.iterrows():
        food_id = row['food_id']
        serving_id = row['serving_id']
        food_name = row['food_name']
        print(f'- Food ID: {food_id}, Serving ID: {serving_id}, Name: {food_name}')

Enter maximum calorie per day: 2000
1/1 [==============================] - 0s 79ms/step
Recommended Breakfast:
- Food ID: 5885612, Serving ID: 5711673, Name: Vegetable & Hummus Wrap
- Food ID: 168435, Serving ID: 211278, Name: Choco Cherry Love Blizzard (Small)
- Food ID: 7828613, Serving ID: 7539190, Name: Battered Onion Rings
- Food ID: 16460374, Serving ID: 15538837, Name: Burrito Breakfast Bowl
- Food ID: 3288608, Serving ID: 3199340, Name: Chicken Lasagna
Recommended Dinner:
- Food ID: 29615443, Serving ID: 27010825, Name: Cranberry Grape
- Food ID: 1549182, Serving ID: 1530242, Name: Extra Dessert Delights Sugarfree Gum - Mint Chocolate Chip
- Food ID: 2222135, Serving ID: 2175204, Name: Buttermilk Bread
- Food ID: 1795583, Serving ID: 1766053, Name: Pomegranate Granola
- Food ID: 5772035, Serving ID: 5603333, Name: Crab Cake
Recommended Lunch:
- Food ID: 26547, Serving ID: 66486, Name: Double-Double Burger
- Food ID: 51764, Serving ID: 103589, Name: Blackened Fish Banzai Bowl
- 